# Monte Carlo Integration

Ref: Chapter 6, Rizzo

## Basic Monte Carlo Integration

Sampling and integartion are important techniques of statistical inference problems. But, often, the target distributions are too complicated and/or the integrands are too complex or high dimensional for these problems to be solved using basic methods.

Let $p(x)$ be a pdf defined on a sample space $X$. Let $h$ be a function mapping $X$ to $\mathbf{R}$.

The goal is to estimate $E[ h(X) ]$. Normally this is an integral computed by hand ($\mu_p(h) = E[ h(x) ] = \int h(x) p(x) dx$). However, we assume that this case is too complicated to solve that way. It depends on the structure of $h$ and $p$ which can be very difficult to calculate.

In general, we will only assume that $\mu_p (|h|) < \infty$. In other words, we assume a finite first moment. 

The **monte carlo method (MCM)** is based on 2 of the most important results in probability theory:

1. Central limit theorem, CLT ($X_n \xrightarrow{} N(\mu, \sigma)$ as $n \xrightarrow{} \infty$)

2. Law of large numbers, LLN ($\bar{x}_n \xrightarrow{} E [ x ]$ as $n \xrightarrow{} \infty$)

The monte carlo method appeals to the LLN and estimates $ E[ h(X) ]$ by the sample mean of $h(x)$: 

$$\bar{h(x)} = \frac{1}{N} \sum_{i=1}^N h(X_i)$$

and $X_1, X_2, ...$ are iid with pdf $p$. This is simply the average.

Recall LLN ensures that $\bar{h(x)} \xrightarrow{} E [ h(x) ]$ as $N \xrightarrow{} \infty$

We can generate RV from $p$ using inverse method (see **Inverse methods** page). This will generate a set of values $h(x_i)$. With these, we can estimate

$\bar{h(x)} = \frac{h(x_1) + h(x_2) + ... + h(x_n)}{N}$

by the LLN. If the estimate is bad, we need a larger $N$.

The confidence inteval can be defined for $E[ h(x)]$ by appealing to the CLT. The sample variance of $\bar{h(x)}$ is

$$Var(\bar{h(x)}) = \frac{\hat{Var}(h(x))}{N} = \frac{1}{N^2} \sum_{i=1}^N (h(x_i) - \bar{h(x)})^2$$

Because (for instance) if $X \sim (\mu, \sigma^2)$, $E[ \bar{x} ] = \mu$ and $Var(\bar{x}) = \frac{\sigma^2}{n}$.

The CLT tells us that the approximate distribution of $\bar{h(x)}$ is approximately normal ($ N(E(h(x)), \frac{var(h(x))}{N})$)

Therefore, the ($1 - \alpha$) CI for $E[ h(x)] is approximately 

$$\bar{h(x)} \pm z_{\alpha/2} \sqrt{\frac{\hat{Var}(h(x))}{N}}$$

**Example:** Estimate $E[ h(x)] = \int_{-\infty}^\infty \frac{1}{1 + \exp (-x)} \frac{1}{\sqrt{ (2\pi) (1.43)}} \exp (\frac{-(x - 1.5)^2}{2.86}) dx$ and give a 95% CI for $E[h(x)]$.

Solving this by hand is hard!

